Open In Colab

%tensorflow_version 2.x
!pip install tensorflow-hub
!pip install transformers
     
Requirement already satisfied: tensorflow-hub in /usr/local/lib/python3.7/dist-packages (0.12.0)
Requirement already satisfied: numpy>=1.12.0 in /usr/local/lib/python3.7/dist-packages (from tensorflow-hub) (1.19.5)
Requirement already satisfied: protobuf>=3.8.0 in /usr/local/lib/python3.7/dist-packages (from tensorflow-hub) (3.12.4)
Requirement already satisfied: setuptools in /usr/local/lib/python3.7/dist-packages (from protobuf>=3.8.0->tensorflow-hub) (57.0.0)
Requirement already satisfied: six>=1.9 in /usr/local/lib/python3.7/dist-packages (from protobuf>=3.8.0->tensorflow-hub) (1.15.0)
Collecting transformers
  Downloading https://files.pythonhosted.org/packages/d5/43/cfe4ee779bbd6a678ac6a97c5a5cdeb03c35f9eaebbb9720b036680f9a2d/transformers-4.6.1-py3-none-any.whl (2.2MB)
     |████████████████████████████████| 2.3MB 4.1MB/s 
Collecting sacremoses
  Downloading https://files.pythonhosted.org/packages/75/ee/67241dc87f266093c533a2d4d3d69438e57d7a90abb216fa076e7d475d4a/sacremoses-0.0.45-py3-none-any.whl (895kB)
     |████████████████████████████████| 901kB 36.1MB/s 
Requirement already satisfied: numpy>=1.17 in /usr/local/lib/python3.7/dist-packages (from transformers) (1.19.5)
Requirement already satisfied: packaging in /usr/local/lib/python3.7/dist-packages (from transformers) (20.9)
Requirement already satisfied: filelock in /usr/local/lib/python3.7/dist-packages (from transformers) (3.0.12)
Requirement already satisfied: regex!=2019.12.17 in /usr/local/lib/python3.7/dist-packages (from transformers) (2019.12.20)
Requirement already satisfied: importlib-metadata; python_version < "3.8" in /usr/local/lib/python3.7/dist-packages (from transformers) (4.0.1)
Collecting tokenizers<0.11,>=0.10.1
  Downloading https://files.pythonhosted.org/packages/d4/e2/df3543e8ffdab68f5acc73f613de9c2b155ac47f162e725dcac87c521c11/tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3MB)
     |████████████████████████████████| 3.3MB 39.2MB/s 
Requirement already satisfied: requests in /usr/local/lib/python3.7/dist-packages (from transformers) (2.23.0)
Requirement already satisfied: tqdm>=4.27 in /usr/local/lib/python3.7/dist-packages (from transformers) (4.41.1)
Collecting huggingface-hub==0.0.8
  Downloading https://files.pythonhosted.org/packages/a1/88/7b1e45720ecf59c6c6737ff332f41c955963090a18e72acbcbeac6b25e86/huggingface_hub-0.0.8-py3-none-any.whl
Requirement already satisfied: six in /usr/local/lib/python3.7/dist-packages (from sacremoses->transformers) (1.15.0)
Requirement already satisfied: joblib in /usr/local/lib/python3.7/dist-packages (from sacremoses->transformers) (1.0.1)
Requirement already satisfied: click in /usr/local/lib/python3.7/dist-packages (from sacremoses->transformers) (7.1.2)
Requirement already satisfied: pyparsing>=2.0.2 in /usr/local/lib/python3.7/dist-packages (from packaging->transformers) (2.4.7)
Requirement already satisfied: zipp>=0.5 in /usr/local/lib/python3.7/dist-packages (from importlib-metadata; python_version < "3.8"->transformers) (3.4.1)
Requirement already satisfied: typing-extensions>=3.6.4; python_version < "3.8" in /usr/local/lib/python3.7/dist-packages (from importlib-metadata; python_version < "3.8"->transformers) (3.7.4.3)
Requirement already satisfied: idna<3,>=2.5 in /usr/local/lib/python3.7/dist-packages (from requests->transformers) (2.10)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /usr/local/lib/python3.7/dist-packages (from requests->transformers) (1.24.3)
Requirement already satisfied: chardet<4,>=3.0.2 in /usr/local/lib/python3.7/dist-packages (from requests->transformers) (3.0.4)
Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.7/dist-packages (from requests->transformers) (2020.12.5)
Installing collected packages: sacremoses, tokenizers, huggingface-hub, transformers
Successfully installed huggingface-hub-0.0.8 sacremoses-0.0.45 tokenizers-0.10.3 transformers-4.6.1


import tensorflow as tf
import tensorflow_hub as hub
from transformers import BertTokenizer


def question_answer(question, reference):
    """                                                                                                                 
    Finds a snippet of text within a reference document to answer a question                                            
                                                                                                                        
    parameters:                                                                                                         
        question [string]:                                                                                              
            contains the question to answer                                                                             
        reference [string]:                                                                                             
            contains the reference document from which to find the answer                                               
                                                                                                                        
    returns:                                                                                                            
        [string]:                                                                                                       
            contains the answer                                                                                         
        or None if no answer is found                                                                                   
    """
    tokenizer = BertTokenizer.from_pretrained(
        'bert-large-uncased-whole-word-masking-finetuned-squad')
    model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")

    quest_tokens = tokenizer.tokenize(question)
    refer_tokens = tokenizer.tokenize(reference)

    tokens = ['[CLS]'] + quest_tokens + ['[SEP]'] + refer_tokens + ['[SEP]']

    input_word_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_word_ids)
    input_type_ids = [0] * (
        1 + len(quest_tokens) + 1) + [1] * (len(refer_tokens) + 1)

    input_word_ids, input_mask, input_type_ids = map(
        lambda t: tf.expand_dims(
            tf.convert_to_tensor(t, dtype=tf.int32), 0),
        (input_word_ids, input_mask, input_type_ids))

    outputs = model([input_word_ids, input_mask, input_type_ids])

    short_start = tf.argmax(outputs[0][0][1:]) + 1
    short_end = tf.argmax(outputs[1][0][1:]) + 1
    answer_tokens = tokens[short_start: short_end + 1]
    answer = tokenizer.convert_tokens_to_string(answer_tokens)


    if answer is None or answer is "" or question in answer:
      return None

    return answer

with open('drive/MyDrive/ZendeskArticles/PeerLearningDays.md') as f:
    reference = f.read()

print(question_answer('jherouhfuoernfnerjnfrn?', reference))
print(question_answer('When are PLDs?', reference))
print(question_answer('What is a potato?', reference))
     
jherouhfuoernfnerjnfrn ?
on - site days from 9 : 00 am to 3 : 00 pm
what is a potato ?

import tensorflow as tf
import tensorflow_hub as hub
from transformers import BertTokenizer


def answer_loop(reference):
    """                                                                                                                 
    Answers questions from a reference text on loop (based on 1-loop.py)                                                
                                                                                                                        
    parameters:                                                                                                         
        reference [string]:                                                                                             
            the reference document from which to find the answer                                                        
    """
    while (1):
        user_input = input("Q: ")
        user_input = user_input.lower()
        if user_input == 'exit' or user_input == 'quit' \
           or user_input == 'goodbye' or user_input == 'bye':
            print("A: Goodbye")
            break
        answer = question_answer(user_input, reference)
        if answer is None:
            print("A: Sorry, I do not understand your question.")
        else:
            print("A: ", answer)


def question_answer(question, reference):
    """                                                                                                                 
    Finds a snippet of text within a reference document to answer a question                                            
                                                                                                                        
    parameters:                                                                                                         
        question [string]:                                                                                              
            contains the question to answer                                                                             
        reference [string]:                                                                                             
            contains the reference document from which to find the answer                                               
                                                                                                                        
    returns:                                                                                                            
        [string]:                                                                                                       
            contains the answer                                                                                         
        or None if no answer is found                                                                                   
    """
    tokenizer = BertTokenizer.from_pretrained(
        'bert-large-uncased-whole-word-masking-finetuned-squad')
    model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")

    quest_tokens = tokenizer.tokenize(question)
    refer_tokens = tokenizer.tokenize(reference)

    tokens = ['[CLS]'] + quest_tokens + ['[SEP]'] + refer_tokens + ['[SEP]']

    input_word_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_word_ids)
    input_type_ids = [0] * (
        1 + len(quest_tokens) + 1) + [1] * (len(refer_tokens) + 1)

    input_word_ids, input_mask, input_type_ids = map(
        lambda t: tf.expand_dims(
            tf.convert_to_tensor(t, dtype=tf.int32), 0),
        (input_word_ids, input_mask, input_type_ids))

    outputs = model([input_word_ids, input_mask, input_type_ids])

    short_start = tf.argmax(outputs[0][0][1:]) + 1
    short_end = tf.argmax(outputs[1][0][1:]) + 1
    answer_tokens = tokens[short_start: short_end + 1]
    answer = tokenizer.convert_tokens_to_string(answer_tokens)

    if answer is None or answer is "" or question in answer:
        return None

    return answer


with open('drive/MyDrive/ZendeskArticles/PeerLearningDays.md') as f:
    reference = f.read()

answer_loop(reference)
     
Q: When are PLDs?
A:  on - site days from 9 : 00 am to 3 : 00 pm
Q: What are Mock Interviews?
A: Sorry, I do not understand your question.
Q: What does PLD stand for?
A:  peer learning days
Q: bye
A: Goodbye

import numpy as np
import os
import tensorflow_hub as hub


def semantic_search(corpus_path, sentence):
    """                                                                                                                 
    Performs semantic search on a corpus of documents                                                                   
                                                                                                                        
    parameters:                                                                                                         
        corpus_path [string]:                                                                                           
            the path to the corpus of reference documents on which                                                      
                to perform semantic search                                                                              
        sentence [string]:                                                                                              
            the sentence from which to perform semantic search                                                          
                                                                                                                        
    returns:                                                                                                            
        [string]:                                                                                                       
            the reference text of the document most similar to given sentence                                           
    """
    documents = [sentence]

    for filename in os.listdir(corpus_path):
        if filename.endswith(".md") is False:
            continue
        with open(corpus_path + "/" + filename, "r", encoding="utf-8") as f:
            documents.append(f.read())

    model = hub.load(
        "https://tfhub.dev/google/universal-sentence-encoder-large/5")

    embeddings = model(documents)

    correlation = np.inner(embeddings, embeddings)

    closest = np.argmax(correlation[0, 1:])

    similar = documents[closest + 1]

    return similar

print(semantic_search('drive/MyDrive/ZendeskArticles', 'When are PLDs?'))
     
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/5'.
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder-large/5, Total size: 577.10MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/5'.
PLD Overview
Peer Learning Days (PLDs) are a time for you and your peers to ensure that each of you understands the concepts you've encountered in your projects, as well as a time for everyone to collectively grow in technical, professional, and soft skills. During PLD, you will collaboratively review prior projects with a group of cohort peers.
PLD Basics
PLDs are mandatory on-site days from 9:00 AM to 3:00 PM. If you cannot be present or on time, you must use a PTO. 
No laptops, tablets, or screens are allowed until all tasks have been whiteboarded and understood by the entirety of your group. This time is for whiteboarding, dialogue, and active peer collaboration. After this, you may return to computers with each other to pair or group program. 
Peer Learning Days are not about sharing solutions. This doesn't empower peers with the ability to solve problems themselves! Peer learning is when you share your thought process, whether through conversation, whiteboarding, debugging, or live coding. 
When a peer has a question, rather than offering the solution, ask the following:
"How did you come to that conclusion?"
"What have you tried?"
"Did the man page give you a lead?"
"Did you think about this concept?"
Modeling this form of thinking for one another is invaluable and will strengthen your entire cohort.
Your ability to articulate your knowledge is a crucial skill and will be required to succeed during technical interviews and through your career. 


import numpy as np
import os
import tensorflow as tf
import tensorflow_hub as hub
from transformers import BertTokenizer


def question_answer(corpus_path):
    """                                                                                        
    Answers questions from multiple reference texts                                            
                                                                                               
    parameters:                                                                                
        corpus_path [string]:                                                                  
            the path to the corpus of reference documents                                      
    """
    while (1):
        user_input = input("Q: ")
        user_input = user_input.lower()
        if user_input == 'exit' or user_input == 'quit' \
           or user_input == 'goodbye' or user_input == 'bye':
            print("A: Goodbye")
            break
        reference = semantic_search(corpus_path, user_input)
        answer = specific_question_answer(user_input, reference)
        if answer is None:
            print("A: Sorry, I do not understand your question.")
        else:
            print("A: ", answer)

def semantic_search(corpus_path, sentence):
    """                                                                                        
    Performs semantic search on a corpus of documents                                          
                                                                                               
    parameters:                                                                                
        corpus_path [string]:                                                                  
            the path to the corpus of reference documents on which                             
                to perform semantic search                                                     
        sentence [string]:                                                                     
            the sentence from which to perform semantic search                                 
                                                                                               
    returns:                                                                                   
        [string]:                                                                              
            the reference text of the document most similar to given sentence                  
    """
    documents = [sentence]

    for filename in os.listdir(corpus_path):
        if filename.endswith(".md") is False:
            continue
        with open(corpus_path + "/" + filename, "r", encoding="utf-8") as f:
            documents.append(f.read())

    model = hub.load(
        "https://tfhub.dev/google/universal-sentence-encoder-large/5")

    embeddings = model(documents)
    correlation = np.inner(embeddings, embeddings)
    closest = np.argmax(correlation[0, 1:])
    similar = documents[closest + 1]

    return similar

def specific_question_answer(question, reference):
    """                                                                                        
    Finds a snippet of text within a reference document to answer a question                   
                                                                                               
    parameters:                                                                                
        question [string]:                                                                     
            contains the question to answer                                                    
        reference [string]:                                                                    
            contains the reference document from which to find the answer                      
                                                                                               
    returns:                                                                                   
        [string]:                                                                              
            contains the answer                                                                
        or None if no answer is found                                                          
    """
    tokenizer = BertTokenizer.from_pretrained(
        'bert-large-uncased-whole-word-masking-finetuned-squad')
    model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")

    quest_tokens = tokenizer.tokenize(question)
    refer_tokens = tokenizer.tokenize(reference)

    tokens = ['[CLS]'] + quest_tokens + ['[SEP]'] + refer_tokens + ['[SEP]']

    input_word_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_word_ids)
    input_type_ids = [0] * (
        1 + len(quest_tokens) + 1) + [1] * (len(refer_tokens) + 1)

    input_word_ids, input_mask, input_type_ids = map(
        lambda t: tf.expand_dims(
            tf.convert_to_tensor(t, dtype=tf.int32), 0),
        (input_word_ids, input_mask, input_type_ids))

    outputs = model([input_word_ids, input_mask, input_type_ids])

    short_start = tf.argmax(outputs[0][0][1:]) + 1
    short_end = tf.argmax(outputs[1][0][1:]) + 1
    answer_tokens = tokens[short_start: short_end + 1]
    answer = tokenizer.convert_tokens_to_string(answer_tokens)
                                                                                               
    if answer is None or answer is "" or question in answer:                                   
        return None                                                                            
                                                                                               
    return answer

question_answer('drive/MyDrive/ZendeskArticles')

     
Q: When are PLDs?
INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/5'.
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder-large/5, Total size: 577.10MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/5'.
HBox(children=(FloatProgress(value=0.0, description='Downloading', max=231508.0, style=ProgressStyle(descripti…
HBox(children=(FloatProgress(value=0.0, description='Downloading', max=28.0, style=ProgressStyle(description_w…
HBox(children=(FloatProgress(value=0.0, description='Downloading', max=466062.0, style=ProgressStyle(descripti…
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/see--/bert-uncased-tf2-qa/1'.
INFO:absl:Downloading https://tfhub.dev/see--/bert-uncased-tf2-qa/1: 544.01MB
INFO:absl:Downloading https://tfhub.dev/see--/bert-uncased-tf2-qa/1: 1.14GB
INFO:absl:Downloaded https://tfhub.dev/see--/bert-uncased-tf2-qa/1, Total size: 1.27GB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/see--/bert-uncased-tf2-qa/1'.
A:  on - site days from 9 : 00 am to 3 : 00 pm
Q: What are Mock Interviews?
A:  help you train for technical interviews
Q: What does PLD stand for?
A:  peer learning days
Q: What is potato?
A:  virtual groups of about 8 students , automatically created , based on a short quiz on study / work habits
Q: What is jkjhrjkhrhkh?
A: Sorry, I do not understand your question.
Q: What does the Chief Kitchen Officer do?
A:  to make sure that the kitchen is maintained to a high standard
Q: exit
A: Goodbye